In [1]:
import rasterio
import numpy as np
import csv
import math
from pathlib import Path

# ----------------------
# Paths
# ----------------------
data_dir = Path("../data")
tiff_path = data_dir / "hyd_na_dem_30s.tif"
nodes_out = "nodes.csv"
edges_out = "edges.csv"

# ----------------------
# Load TIFF
# ----------------------
with rasterio.open(tiff_path) as src:
    nodata = src.nodata
    arr = src.read(1)

rows, cols = arr.shape
print(f"Raster shape: {rows} x {cols}")
print(f"NoData value: {nodata}")

# ----------------------
# Neighbor offsets (8-connected)
# ----------------------
neighbors = [
    (-1, 0), (1, 0), (0, -1), (0, 1),      # orthogonal
    (-1, -1), (-1, 1), (1, -1), (1, 1)     # diagonal
]

Raster shape: 6960 x 10320
NoData value: 32767.0


In [2]:
# ----------------------
# Write nodes.csv
# ----------------------
node_map = {}  # (row, col) -> node_id
node_id = 0

with open(nodes_out, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "label", "row", "col", "value"])
    
    for r in range(rows):
        for c in range(cols):
            val = int(arr[r, c])
            if val == nodata:
                continue  # skip NoData
            writer.writerow([node_id, node_id, r, c, val])
            node_map[(r, c)] = node_id
            node_id += 1

print("Finished writing {len(node_map)} nodes to {nodes_out}")


Finished writing 25670141 nodes to nodes.csv


In [ ]:
# ----------------------
# Write edges.csv
# ----------------------
with open(edges_out, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["source", "target", "weight"])

    edge_count = 0
    for (r, c), src_id in node_map.items():
        for dr, dc in neighbors:
            rr, cc = r + dr, c + dc
            if (rr, cc) in node_map:  # only connect valid neighbors
                dst_id = node_map[(rr, cc)]
                weight = math.sqrt(2) if abs(dr) == 1 and abs(dc) == 1 else 1
                writer.writerow([src_id, dst_id, weight])
                edge_count += 1

print("Finished writing {edge_count} edges to {edges_out}")

In [15]:
# ----------------------
# Troubleshooting
# ----------------------
import rasterio

with rasterio.open("../data/hyd_na_dem_30s.tif") as src:
    arr = src.read(1)
    print("Raster shape:", arr.shape)
    print("Min/Max values:", arr.min(), arr.max())

with rasterio.open("../data/hyd_na_dem_30s.tif") as src:
    print("nodata value:", src.nodata)
    arr = src.read(1)
    print("Unique values:", np.unique(arr))

Raster shape: (6960, 10320)
Min/Max values: -138 32767
